In [1]:
import os
os.chdir('/home/smallyan/eval_agent')

# Check torch version after upgrade
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

PyTorch version: 2.9.1+cu128
CUDA available: True
GPU: NVIDIA A100 80GB PCIe


# Circuit Analysis Code Evaluation

This notebook evaluates the code implementation in `/net/scratch2/smallyan/relations_eval` for circuit analysis (Linear Relational Embeddings).

## Setup

In [2]:
repo_path = '/net/scratch2/smallyan/relations_eval'

# Add repo to path
import sys
sys.path.insert(0, repo_path)

# Initialize evaluation tracking
evaluation_results = []
corrections_made = 0
blocks_that_failed = 0

def record_evaluation(block_id, runnable, correct_impl, redundant, irrelevant, error_note=""):
    """Record evaluation for a code block"""
    global blocks_that_failed, corrections_made
    evaluation_results.append({
        'block_id': block_id,
        'runnable': runnable,
        'correct_implementation': correct_impl,
        'redundant': redundant,
        'irrelevant': irrelevant,
        'error_note': error_note
    })
    if not runnable or not correct_impl:
        blocks_that_failed += 1
    
print("Evaluation tracking initialized")
print(f"Repository path: {repo_path}")

Evaluation tracking initialized
Repository path: /net/scratch2/smallyan/relations_eval


## Evaluating demo/demo.ipynb

This is the main demo notebook showing LRE approximation, faithfulness and causality metrics.

In [3]:
# Block 1: demo/demo.ipynb - Cell 1 (Imports)
block_id = "demo/demo.ipynb:cell_1_imports"
try:
    import torch
    from src import models, data, lens, functional
    from src.utils import experiment_utils
    runnable = True
    error_note = ""
except Exception as e:
    runnable = False
    error_note = str(e)

record_evaluation(block_id, runnable=runnable, correct_impl=True, redundant=False, irrelevant=False, error_note=error_note)
print(f"Block: {block_id}")
print(f"  Runnable: {'Y' if runnable else 'N'}")
if error_note:
    print(f"  Error: {error_note}")

Block: demo/demo.ipynb:cell_1_imports
  Runnable: N
  Error: Failed to import transformers.models.gpt2.modeling_gpt2 because of the following error (look up to see its traceback):
operator torchvision::nms does not exist


In [4]:
# The issue is with torchvision being incompatible. Let's fix it
import subprocess

result = subprocess.run(
    ['pip', 'install', 'torchvision>=0.18.0', '--quiet'],
    capture_output=True,
    text=True
)
print(f"Torchvision upgrade: {result.returncode}")
if result.stderr:
    print(f"stderr: {result.stderr[:500]}")

Torchvision upgrade: 0
stderr:   WARNING: Failed to remove contents in a temporary directory '/home/smallyan/.conda/envs/meta/lib/python3.11/site-packages/nvidia/~vjitlink'.
  You can safely remove it manually.
  You can safely remove it manually.


In [5]:
# Try importing again after the upgrade - need to restart kernel for this
# Let me try a fresh import approach

# Clear any cached modules
import sys
mods_to_clear = [k for k in sys.modules.keys() if 'torchvision' in k]
for m in mods_to_clear:
    del sys.modules[m]

# Now try importing transformers
try:
    import transformers
    print(f"Transformers version: {transformers.__version__}")
    from transformers import GPT2LMHeadModel
    print("GPT2LMHeadModel imported successfully")
except Exception as e:
    print(f"Error: {e}")

Transformers version: 4.46.1
Error: Failed to import transformers.models.gpt2.modeling_gpt2 because of the following error (look up to see its traceback):
operator torchvision::nms does not exist
